In [ ]:
from utils import load_files
from main import transcribe

In [ ]:
model , decoder = load_files()

In [ ]:
transcript = transcribe("test.flac", model, decoder)

In [ ]:
print(transcript)

In [ ]:
import sounddevice as sd
import soundfile as sf


def record_audio(duration, sample_rate=16000, file_path="test.wav"):
    # Set the sample rate and duration
    duration_in_samples = int(duration * sample_rate)

    # Record audio
    audio = sd.rec(frames=duration_in_samples,
                   samplerate=sample_rate, channels=1, blocking=True)

    # Save audio to file
    sf.write(file_path, audio, sample_rate)

    return file_path


# Example usage: Record audio for 5 seconds at a sample rate of 16000 Hz and save to "recorded_audio.wav"
audio_file = record_audio(duration=2, sample_rate=16000,
                          file_path="test.wav")
print("Audio saved to:", audio_file)


In [ ]:
transcript = transcribe("test.wav", model, decoder)
print(transcript)